## Load packages for the project...

In [17]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

import itertools
import cPickle

## Load sqlite database...

In [2]:
cd ~/datacourse/Capstone/CollegeScorecard_Raw_Data/

/home/vagrant/datacourse/Capstone/CollegeScorecard_Raw_Data


In [3]:
conn = sqlite3.connect('database.sqlite')

In [4]:
sqlstr = "SELECT count(*) FROM Scorecard"
pd.read_sql(sqlstr, conn)

,count(*)
0,124699


## Table of Variables

 Variable Name  | What it means?                                                                              
 :-----------:|------------------------------------------------------------------------------------------- 
 NPT4_PRIV    | Average net price for Title IV institutions (private for-profit and nonprofit institutions) 
 YEAR         | Academic year 
 UNITID | UnivID
  INSTNM    | College,\
  CONTROL   | CollegeType,\
  COSTT4_A  |Cost,\
  TUITFTE    |TuitionFTE,\
  INEXPFTE   |ExpenditureFTE,\
  AVGFACSAL  |AvgFacultySal,\
  UGDS       |UGEnrollment,\
  SATMTMID   |Math,\
  SATVRMID   |Verbal,\
  SATWRMID   |Writing,\
  PCTPELL    |PercentPell,\
  DEBT_MDN   |MedianDebt,\
  INC_PCT_M1 |
  INC_PCT_M2 |
  INC_PCT_H1 |
  INC_PCT_H2 |
  mn_earn_wne_p10 | E50

## Admission rates...

In [5]:
admRateStr = "SELECT INSTNM College,\
                ADM_RATE*100.0 AdmissionRate\
                FROM Scorecard\
                WHERE Year=2013\
                AND ADM_RATE IS NOT NULL\
                AND ADM_RATE != 0.0\
                AND PREDDEG='Predominantly bachelor''s-degree granting'\
                AND CCBASIC NOT LIKE '%Special Focus%'\
                ORDER BY ADM_RATE\
                LIMIT 20"

adminRateDF = pd.read_sql(admRateStr, conn)

In [6]:
cPickle.dump(adminRateDF, open('adminRateDF.pkl', 'wb'))

## Data available for school years....

In [18]:
sqlstr = "SELECT DISTINCT Year FROM Scorecard"
pd.read_sql(sqlstr, conn)

,Year
0,1996
1,1997
2,1998
3,1999
4,2000
5,2001
6,2002
7,2003
8,2004
9,2005


## Number of schools over the years...

In [6]:
noOfSchoolsStr =  "SELECT Year, COUNT(Id) NumSchools FROM Scorecard GROUP BY Year"
noOfSchools = pd.read_sql(noOfSchoolsStr, conn)

In [7]:
cPickle.dump(noOfSchools, open('noOfSchools.pkl', 'wb'))

## Figure out the geo-coordinates of all schools...

In [8]:
sqlstr = "SELECT    UNITID, \
                    INSTNM, \
                    CITY, \
                    STABBR, \
                    ZIP, \
                    PREDDEG, \
                    CURROPER,\
                    TUITIONFEE_IN, \
                    LATITUDE, \
                    LONGITUDE, \
                    DISTANCEONLY,\
                    DEATH_YR4_RT, \
                    COMP_ORIG_YR4_RT, \
                    GRAD_DEBT_MDN \
          FROM Scorecard" 
schoolsDF = pd.read_sql(sqlstr, conn)

In [9]:
cPickle.dump(schoolsDF, open('schoolsDF.pkl', 'wb'))

## School address and geolocation...

In [ ]:
sqlstr = "SELECT    UNITID, \
                    INSTNM, \
                    CITY, \
                    STABBR, \
                    ZIP \
          FROM Scorecard" 
schoolAddress = pd.read_sql(sqlstr, conn)

In [ ]:
schoolAddress.to_csv('schoolAddress.csv', encoding='utf-8')

In [ ]:
schoolLoc = pd.read_csv('schoolAddress.csv')

In [ ]:
def zipCode(string):
    return string[0:5]
schoolLoc['ZIP'] = schoolLoc['ZIP'].apply(zipCode)

In [ ]:
schoolLocList = pd.DataFrame(list(schoolLoc['INSTNM'] +', '
                              + schoolLoc['CITY'] +', '
                              + schoolLoc['STABBR']+', '
                              + schoolLoc['ZIP']))

In [ ]:
address = []
for i in range(len(schoolLocList)):
    address.append(schoolLocList.values[i][0])
    print str(schoolLocList.values[i][0])
    if i > 20:
        break

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## School enrollment

In [10]:
enrollmentStr = "SELECT INSTNM College, \
                 UGDS UndergradEnrollment,\
                 CONTROL CollegeType\
                 FROM Scorecard\
                 WHERE Year = 2013\
                 AND PREDDEG = 'Predominantly bachelor''s-degree granting'\
                 AND CCBASIC NOT LIKE '%Special Focus%'\
                 AND UGDS IS NOT NULL\
                 AND UGDS > 0\
                 ORDER BY UGDS DESC"
enrollment = pd.read_sql(enrollmentStr, conn)

In [11]:
cPickle.dump(enrollment, open('enrollment.pkl', 'wb'))

## College cost...

In [12]:
costSqlStr = "SELECT INSTNM College,\
                COSTT4_A Cost, \
                CONTROL CollegeType\
                FROM Scorecard\
                WHERE Year=2013\
                AND PREDDEG='Predominantly bachelor''s-degree granting'\
                AND CCBASIC NOT LIKE '%Special Focus%'\
                AND COSTT4_A IS NOT NULL\
                ORDER BY COSTT4_A DESC"
collegeCost = pd.read_sql(costSqlStr, conn)

In [13]:
cPickle.dump(collegeCost, open('collegeCost.pkl', 'wb'))

## Median earnings once they graduate...

In [14]:
earningsStr = "SELECT s11.INSTNM College,\
               s11.CONTROL CollegeType,\
               s11.md_earn_wne_p10 e50,\
               s11.pct10_earn_wne_p10 e10,\
               s11.pct25_earn_wne_p10 e25,\
               s11.pct75_earn_wne_p10 e75,\
               s11.pct90_earn_wne_p10 e90\
            FROM Scorecard s11\
            INNER JOIN Scorecard s13 ON s11.UNITID=s13.UNITID\
            WHERE s11.Year=2011\
              AND s13.Year=2013\
              AND s11.pct75_earn_wne_p10 IS NOT NULL\
              AND s11.pct75_earn_wne_p10 != 'PrivacySuppressed'\
              AND s11.PREDDEG = 'Predominantly bachelor''s-degree granting'\
              AND s13.CCBASIC NOT LIKE '%Special%'\
            ORDER BY s11.pct75_earn_wne_p10 DESC"

earnings = pd.read_sql(earningsStr, conn)

In [15]:
cPickle.dump(earnings, open('earnings.pkl', 'wb'))

##SAT Performance...

In [16]:
SATstr = "SELECT INSTNM College,\
           SATMTMID Math,\
           SATVRMID Verbal,\
           SATWRMID Writing\
        FROM Scorecard\
        WHERE Year=2011\
          AND SATMTMID IS NOT NULL\
          AND SATMTMID != 'PrivacySuppressed'\
          AND SATVRMID IS NOT NULL\
          AND SATVRMID != 'PrivacySuppressed'\
          AND SATWRMID IS NOT NULL\
          AND SATWRMID != 'PrivacySuppressed'"
SATPerformance = pd.read_sql(SATstr, conn)

In [17]:
cPickle.dump(SATPerformance, open('SATPerformance.pkl', 'wb'))

## Pell grant to median income correlation...

In [18]:
PellStr = "SELECT INSTNM College,\
                  PCTPELL  PercentPell,\
                  md_earn_wne_p10 e50\
            FROM Scorecard\
            WHERE Year=2011\
                AND PCTPELL IS NOT NULL\
                AND md_earn_wne_p10 != 'PrivacySuppressed'"
PellMatrix = pd.read_sql(PellStr, conn)

In [19]:
cPickle.dump(PellMatrix, open('PellMatrix.pkl', 'wb'))

## Completion rate analysis...

In [49]:
completionStr = "SELECT INSTNM                     College,\
                        Year                       Year,\
                        C150_4                     completion_rate_4yr_150nt\
                 FROM Scorecard"
CompletionRate = pd.read_sql(completionStr, conn)

In [56]:
CompletionRate.dropna(how='any')
CompletionRate.dropna()

,College,Year,completion_rate_4yr_150nt
6795,ALABAMA A & M UNIVERSITY,1997,0.3455
6796,UNIVERSITY OF ALABAMA AT BIRMINGHAM,1997,0.3319
6799,UNIVERSITY OF ALABAMA IN HUNTSVILLE,1997,0.3333
6800,ALABAMA STATE UNIVERSITY,1997,0.1785
6801,THE UNIVERSITY OF ALABAMA,1997,0.5735
6805,AUBURN UNIVERSITY MAIN CAMPUS,1997,0.6505
6810,CONCORDIA COLLEGE,1997,0.1739
6826,HERZING COLLEGE,1997,0.5356
6827,HUNTINGDON COLLEGE,1997,0.4392
6828,INTERNATIONAL BIBLE COLLEGE,1997,0.4000


## Collect the data for the final model...

In [25]:
modelStr = "SELECT UNITID UnivID,\
                  INSTNM     College,\
                  CONTROL    CollegeType,\
                  COSTT4_A   Cost,\
                  TUITFTE    TuitionFTE,\
                  INEXPFTE   ExpenditureFTE,\
                  AVGFACSAL  AvgFacultySal,\
                  UGDS       UGEnrollment,\
                  SATMTMID   Math,\
                  SATVRMID   Verbal,\
                  SATWRMID   Writing,\
                  PCTPELL    PercentPell,\
                  DEBT_MDN   MedianDebt,\
                  INC_PCT_M1,\
                  INC_PCT_M2,\
                  INC_PCT_H1,\
                  INC_PCT_H2,\
                  mn_earn_wne_p10 E50\
           FROM  Scorecard\
           WHERE YEAR = 2011\
           AND   CURROPER != 'Not currently certified as an operating institution'\
           AND   PREDDEG = 'Predominantly bachelor''s-degree granting'\
           AND   SATMTMID IS NOT NULL\
           AND   SATMTMID != 'PrivacySuppressed'\
           AND   SATVRMID IS NOT NULL\
           AND   SATVRMID != 'PrivacySuppressed'\
           AND   SATWRMID IS NOT NULL\
           AND   SATWRMID != 'PrivacySuppressed'\
           AND   PCTPELL IS NOT NULL\
           AND   md_earn_wne_p10 != 'PrivacySuppressed'\
           AND   INC_PCT_M1 IS NOT NULL\
           AND   INC_PCT_M1 != 'PrivacySuppressed'\
           AND   INC_PCT_M2 IS NOT NULL\
           AND   INC_PCT_M2 != 'PrivacySuppressed'\
           AND   INC_PCT_H1 IS NOT NULL\
           AND   INC_PCT_H1 != 'PrivacySuppressed'\
           AND   INC_PCT_H2 IS NOT NULL\
           AND   INC_PCT_H2 != 'PrivacySuppressed'"
           
        
modelData = pd.read_sql(modelStr, conn)

In [26]:
modelData.describe()

,UnivID,Cost,TuitionFTE,ExpenditureFTE,AvgFacultySal,UGEnrollment,Math,Verbal,Writing,PercentPell,MedianDebt,INC_PCT_M1,INC_PCT_M2,INC_PCT_H1,INC_PCT_H2,E50
count,427.000000,427.000000,412.000000,412.000000,426.000000,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000
mean,180784.227166,32554.037471,12771.577670,10048.378641,7197.002347,4276.704918,531.988290,526.843091,516.147541,0.363575,16807.750585,0.149288,0.169109,0.156432,0.177997,48812.177986
std,40813.268899,11909.723286,6276.650563,8259.316545,1848.839577,6145.828595,75.261811,70.167748,75.362749,0.143104,3841.041372,0.028067,0.032836,0.044651,0.099901,14029.601224
min,100858.000000,8986.000000,1969.000000,1324.000000,3364.000000,230.000000,355.000000,365.000000,350.000000,0.078500,5000.000000,0.078925,0.066667,0.015560,0.000000,26400.000000
25%,151319.500000,21362.000000,7731.750000,6263.500000,5923.500000,1341.500000,485.000000,480.000000,465.000000,0.265550,14409.500000,0.132881,0.147305,0.129842,0.095822,39850.000000
50%,183062.000000,32853.000000,12259.000000,7914.000000,6813.500000,2190.000000,520.000000,515.000000,505.000000,0.353300,17125.000000,0.147423,0.168269,0.160572,0.160759,46000.000000
75%,213906.500000,40798.000000,16526.500000,10522.750000,8140.500000,4521.000000,567.500000,565.000000,555.000000,0.442900,19500.000000,0.167222,0.188731,0.186025,0.243419,53500.000000
max,433660.000000,56641.000000,31562.000000,84831.000000,15866.000000,66864.000000,780.000000,745.000000,745.000000,0.891000,25000.000000,0.273834,0.282642,0.288433,0.452468,128400.000000


In [44]:
modelStr = "SELECT UNITID UnivID,\
                  INSTNM     College,\
                  CONTROL    CollegeType,\
                  COSTT4_A   Cost,\
                  TUITFTE    TuitionFTE,\
                  INEXPFTE   ExpenditureFTE,\
                  AVGFACSAL  AvgFacultySal,\
                  UGDS       UGEnrollment,\
                  SATMTMID   Math,\
                  SATVRMID   Verbal,\
                  SATWRMID   Writing,\
                  PCTPELL    PercentPell,\
                  DEBT_MDN   MedianDebt,\
                  NPT4_PRIV  AvgNetPricePrivate,\
                  INC_PCT_M1,\
                  INC_PCT_M2,\
                  INC_PCT_H1,\
                  INC_PCT_H2,\
                  mn_earn_wne_p10 E50,\
                  md_earn_wne_p10 M50\
           FROM  Scorecard\
           WHERE YEAR = 2011\
           AND   CURROPER != 'Not currently certified as an operating institution'\
           AND   PREDDEG = 'Predominantly bachelor''s-degree granting'\
           AND   SATMTMID IS NOT NULL\
           AND   SATMTMID != 'PrivacySuppressed'\
           AND   SATVRMID IS NOT NULL\
           AND   SATVRMID != 'PrivacySuppressed'\
           AND   SATWRMID IS NOT NULL\
           AND   SATWRMID != 'PrivacySuppressed'\
           AND   PCTPELL IS NOT NULL\
           AND   md_earn_wne_p10 != 'PrivacySuppressed'\
           AND   INC_PCT_M1 IS NOT NULL\
           AND   INC_PCT_M1 != 'PrivacySuppressed'\
           AND   INC_PCT_M2 IS NOT NULL\
           AND   INC_PCT_M2 != 'PrivacySuppressed'\
           AND   INC_PCT_H1 IS NOT NULL\
           AND   INC_PCT_H1 != 'PrivacySuppressed'\
           AND   INC_PCT_H2 IS NOT NULL\
           AND   INC_PCT_H2 != 'PrivacySuppressed'"
           
        
modelData = pd.read_sql(modelStr, conn)

In [45]:
modelData.head()

,UnivID,College,CollegeType,Cost,TuitionFTE,ExpenditureFTE,AvgFacultySal,UGEnrollment,Math,Verbal,Writing,PercentPell,MedianDebt,AvgNetPricePrivate,INC_PCT_M1,INC_PCT_M2,INC_PCT_H1,INC_PCT_H2,E50,M50
0,100858,Auburn University,Public,21140,11043,10823,8808,20436,570,550,540,0.1767,17625,NaN,0.112864,0.152121,0.176831,0.296004,52100,45400
1,100937,Birmingham Southern College,Private nonprofit,39848,12707,9730,6745,1305,560,560,560,0.2412,18750,18928,0.140553,0.188940,0.175115,0.262673,45300,41900
2,101189,Faulkner University,Private nonprofit,25198,10041,4750,6747,2689,363,382,352,0.5801,18750,18597,0.140915,0.108292,0.090621,0.042139,43500,37600
3,101435,Huntingdon College,Private nonprofit,29369,8993,6254,5385,1110,506,518,477,0.4056,15000,16662,0.150000,0.184211,0.134211,0.151316,40300,37100
4,101541,Judson College,Private nonprofit,26090,5022,7617,5775,341,460,495,480,0.4348,12480,16012,0.211538,0.168269,0.110577,0.072115,33800,30700


## Get the geo-coordinates of the schools...

In [ ]:
schools2013LatLong = schoolsDF2013[['UNITID',
                                    'INSTNM',
                                    'LATITUDE',       
                                    'LONGITUDE',      
                                    'DISTANCEONLY',   
                                    'DEATH_YR4_RT',   
                                    'COMP_ORIG_YR4_RT', 
                                    'GRAD_DEBT_MDN']]

In [ ]:
schools2013LatLong1 = schoolsDF2013[['LATITUDE',
                                     'LONGITUDE',
                                     'INSTNM']]

In [ ]:
schools2013LatLong1 = schools2013LatLong1.replace('nan', np.nan)
schools2013LatLong1 = schools2013LatLong1.dropna(axis=0, how='any')
len(schools2013LatLong1)

In [ ]:
import codecs

fhand = codecs.open('where.js','w', "utf-8")
fhand.write("myData = [\n")
count = 0

for row in itertools.izip(schools2013LatLong1['LATITUDE'], schools2013LatLong1['LONGITUDE'], schools2013LatLong1['INSTNM']):   

    try : 
        count = count + 1
        if count > 1 : fhand.write(",\n")
        output = '['+str(row[0])+', '+str(row[1])+', "'+ row[2]+'"]'
        fhand.write(output)
    except:
        continue

fhand.write("\n];\n")